In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score

In [4]:
# Load the dataset from 'merged.csv'
data = pd.read_csv('/Users/PRADEEP SHETTY/Desktop/IEEE internship/merged_final.csv')

In [27]:
# Convert the "timestamp" column to a pandas datetime object
data['timestamp'] = pd.to_datetime(data['date'] + ' ' + data['hour'].apply(lambda x: '{:02d}'.format(x)), format='%Y-%m-%d %H')

In [28]:
# Sort the data based on the timestamp index
data = data.sort_values(by='timestamp')

In [29]:
# Extract features and labels
X_datetime_features = (data['timestamp'] - data['timestamp'].iloc[0]).dt.total_seconds().values.reshape(-1, 1)
X_activity = data[['Avg_activity']].values
y = data['status'].values

In [30]:
# Normalize the activity feature
#activity_scaler = MinMaxScaler()
#X_activity_normalized = activity_scaler.fit_transform(X_activity)

In [31]:
# Encode the labels (0 for control and 1 for condition)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [32]:
# Set parameters for the model
window_size = 10      # Define the number of time steps to consider as input
num_features_datetime = 1  # Only one feature: 'timestamp'
num_features_activity = 1  # Only one feature: 'Avg_activity'

In [33]:
# Combine datetime and normalized 'Avg_activity' features
X_combined = np.concatenate((X_datetime_features, X_activity), axis=1)

In [34]:
# Discard the remaining data points that cannot form a complete window
num_samples = len(X_combined) - (len(X_combined) % window_size)
X_combined = X_combined[:num_samples]
y_encoded = y_encoded[:num_samples]

In [35]:
# Reshape input features to match the input shape expected by LSTM (samples, time steps, features)
X_combined = X_combined.reshape((-1, window_size, num_features_datetime + num_features_activity))

In [36]:
# Ensure both features and labels have the same number of samples
y_encoded = y_encoded[:len(X_combined)]

In [37]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_encoded, test_size=0.2, random_state=42)

In [38]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(window_size, num_features_datetime + num_features_activity)))
model.add(Dense(1, activation='sigmoid'))

In [39]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
# Train the model
num_epochs = 50      # Define the number of epochs for training
batch_size = 32      # Define the batch size
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size)

Epoch 1/50
66/66 [==============================] - 2s 6ms/step - loss: 0.6402 - accuracy: 0.6621
Epoch 2/50
66/66 [==============================] - 0s 5ms/step - loss: 0.6399 - accuracy: 0.6621
Epoch 3/50
66/66 [==============================] - 0s 5ms/step - loss: 0.6400 - accuracy: 0.6621
Epoch 4/50
66/66 [==============================] - 0s 5ms/step - loss: 0.6399 - accuracy: 0.6621
Epoch 5/50
66/66 [==============================] - 0s 5ms/step - loss: 0.6400 - accuracy: 0.6621
Epoch 6/50
66/66 [==============================] - 0s 5ms/step - loss: 0.6400 - accuracy: 0.6621
Epoch 7/50
66/66 [==============================] - 0s 5ms/step - loss: 0.6400 - accuracy: 0.6621
Epoch 8/50
66/66 [==============================] - 0s 5ms/step - loss: 0.6400 - accuracy: 0.6621
Epoch 9/50
66/66 [==============================] - 0s 6ms/step - loss: 0.6399 - accuracy: 0.6621
Epoch 10/50
66/66 [==============================] - 0s 5ms/step - loss: 0.6398 - accuracy: 0.6621
Epoch 11/50
66/66 [

In [46]:
# Make predictions on the test set
y_pred_probabilities = model.predict(X_test)
y_pred_binary = (y_pred_probabilities >= 0.5).astype(int).flatten()


17/17 [==============================] - 0s 3ms/step


In [47]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.6723809523809524


In [48]:
# Inverse transform the labels to get the original 'status' values
actual_labels = label_encoder.inverse_transform(y_test)
predicted_labels = label_encoder.inverse_transform(y_pred_binary)



In [49]:
# Inverse transform the labels to get the original 'status' values
#actual_labels = label_encoder.inverse_transform(y_test)
#predicted_labels = label_encoder.inverse_transform(y_pred_binary.astype(int))

In [51]:
# Print the actual and predicted values side by side
print("Timestamp\t\t\tActual Status\t\tPredicted Status")
print("------------------------------------------------------")
for i, timestamp in enumerate(data['timestamp'][-len(X_test):]):
    print(f"{timestamp}\t{actual_labels[i]}\t\t\t{predicted_labels[i]}")


Timestamp			Actual Status		Predicted Status
------------------------------------------------------
2006-01-30 14:00:00	1			1
2006-01-30 14:00:00	0			1
2006-01-30 15:00:00	1			1
2006-01-30 15:00:00	1			1
2006-01-30 16:00:00	1			1
2006-01-30 16:00:00	1			1
2006-01-30 17:00:00	0			1
2006-01-30 17:00:00	0			1
2006-01-30 18:00:00	1			1
2006-01-30 18:00:00	1			1
2006-01-30 19:00:00	0			1
2006-01-30 19:00:00	1			1
2006-01-30 20:00:00	0			1
2006-01-30 20:00:00	1			1
2006-01-30 21:00:00	1			1
2006-01-30 21:00:00	1			1
2006-01-30 22:00:00	1			1
2006-01-30 22:00:00	1			1
2006-01-30 23:00:00	1			1
2006-01-30 23:00:00	0			1
2006-01-31 00:00:00	0			1
2006-01-31 00:00:00	1			1
2006-01-31 01:00:00	1			1
2006-01-31 01:00:00	1			1
2006-01-31 02:00:00	1			1
2006-01-31 02:00:00	1			1
2006-01-31 03:00:00	1			1
2006-01-31 03:00:00	1			1
2006-01-31 04:00:00	1			1
2006-01-31 04:00:00	0			1
2006-01-31 05:00:00	1			1
2006-01-31 05:00:00	1			1
2006-01-31 06:00:00	1			1
2006-01-31 06:00:00	1			1
2006-01-31 07:00:

In [44]:
# Making Predictions and Printing Actual and Predicted Values
#y_pred_one_hot = model.predict(X_test)
#y_pred = np.argmax(y_pred_one_hot, axis=1)
#predicted_labels = label_encoder.inverse_transform(y_pred)
#actual_labels = label_encoder.inverse_transform(np.argmax(y_test, axis=1))

#print("Timestamp\t\t\tActual Status\t\tPredicted Status")
#for i, timestamp in enumerate(timestamps[-len(X_test):]):
 #   print(f"{timestamp}\t{actual_labels[i]}\t\t\t{predicted_labels[i]}")

In [45]:
# Print the actual and predicted values side by side
print("Timestamp\t\t\tActual Status\t\tPredicted Status")
print("-------------------------------------------------------------------------")
for i, timestamp in enumerate(data['timestamp'][-len(X_test):]):

    print(f"{timestamp}\t\t\t{actual_labels[i]}\t\t\t{predicted_labels[i]}")

Timestamp			Actual Status		Predicted Status
-------------------------------------------------------------------------
2006-01-30 14:00:00			1			1
2006-01-30 14:00:00			0			1
2006-01-30 15:00:00			1			1
2006-01-30 15:00:00			1			1
2006-01-30 16:00:00			1			1
2006-01-30 16:00:00			1			1
2006-01-30 17:00:00			0			1
2006-01-30 17:00:00			0			1
2006-01-30 18:00:00			1			1
2006-01-30 18:00:00			1			1
2006-01-30 19:00:00			0			1
2006-01-30 19:00:00			1			1
2006-01-30 20:00:00			0			1
2006-01-30 20:00:00			1			1
2006-01-30 21:00:00			1			1
2006-01-30 21:00:00			1			1
2006-01-30 22:00:00			1			1
2006-01-30 22:00:00			1			1
2006-01-30 23:00:00			1			1
2006-01-30 23:00:00			0			1
2006-01-31 00:00:00			0			1
2006-01-31 00:00:00			1			1
2006-01-31 01:00:00			1			1
2006-01-31 01:00:00			1			1
2006-01-31 02:00:00			1			1
2006-01-31 02:00:00			1			1
2006-01-31 03:00:00			1			1
2006-01-31 03:00:00			1			1
2006-01-31 04:00:00			1			1
2006-01-31 04:00:00			0			1
2006-01-31 05:00:00			1			1
2006-01-31 05: